<a href="https://colab.research.google.com/github/ContiPaolo/Multifidelity-Tutorial/blob/main/MF_POD_Burger's1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Comparison of the models without fenics

#### Import libraries

In [26]:
import numpy as np
#import matplotlib as plt
from matplotlib import pyplot as plt
import pandas as pd 
import timeit

from tensorflow.keras.optimizers import Adam,Nadam,Adamax
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, Add
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import load_model
from sklearn.utils import extmath
from sklearn.model_selection import KFold

from tensorflow.keras.regularizers import l2


#######################     CONFIGURATIONS     ##########################
seed = 29
train = True
save = True

In [27]:
# Extract test data for visualization or further processing
n_eig = 64
X = np.loadtxt('../../data/50-25-10/X_test_50resolution.csv', delimiter = ',')
y = np.loadtxt('../../data/50-25-10/y_test_50resolution.csv',delimiter = ',')

Load low fidelity model

In [28]:
# Choose the model parameters 
n_samples_lf = 64000
coeff_lf = 1e-09
# Choose the model parameters 
n_samples = 16000
coeff = 1e-08
# Initialize the neural network model
model_l = Sequential([
    Dense(256, input_shape=(X.shape[1],), activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(25, activation='linear')
])

model_l = load_model(f'models/model_25resolution_{n_samples_lf}samples_1_coeff{coeff_lf}.keras')

n_neurons = 256
# Initialize the neural network model
# Define the three branches of the model
input_params = Input(shape=(X.shape[1],))
input_pod = Input(shape=(y.shape[1],))

# Define the first branch (parameters)
x1 = Dense(n_neurons, activation='gelu')(input_params)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)

# Define the second branch (POD)
x2 = Dense(n_neurons, activation='gelu')(input_pod)

# # Define the second branch (POD)
# x3 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(w))(input_nn)

# Combine the outputs of the three branches
combined = Add()([x1,x2])
combined = Dense(n_neurons, activation='gelu')(combined)
output = Dense(25, activation='linear')(combined)

# Create the model
model_h = Model(inputs=[input_params,input_pod], outputs=output)


model_h = load_model(f'models/model_2step_50-25resolution_{n_samples}samples_1_coeff{coeff}.keras')

model_lf = lambda input : model_l(input.reshape(1,64)).numpy().reshape(25)
model_hf = lambda input: model_h([input.reshape(1,64), model_l(input.reshape(1,64)).numpy()]).numpy().reshape(25)

In [37]:
start_0 = timeit.default_timer() 
fine_predictions = [model_lf(X[1]) for i in range(10000)]
end_0 = timeit.default_timer()

start_1 = timeit.default_timer() 
fine_predictions = [model_hf(X[1]) for i in range(10000)]
end_1= timeit.default_timer()


print('Time Low Fidelity', (end_0-start_0)/10000)
print('Time High Fidelity', (end_1-start_1)/10000)
print('The High/Low : ', (end_1 - start_1)/(end_0 - start_0) )

Time Low Fidelity 0.002644597616700048
Time High Fidelity 0.006013955195900053
The High/Low :  2.2740530196061806


# FROM HERE ON USE FENICS ENVIRONMENT

In [2]:
# Standard library imports
import sys

import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['KERAS_BACKEND'] = 'tensorflow'

# Third-party library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import arviz as az
import timeit

import scipy.stats as stats
from keras.models import Model as Model_nn
from keras.models import Sequential, load_model
from keras.layers import Dense, Concatenate
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Input, Dense, Add

#Try with TinyDA
import tinyDA as tda
from scipy.stats import multivariate_normal
from scipy.stats import uniform
from itertools import product

# Local module imports
sys.path.append('../../')
sys.path.append('../../solver')
#sys.path.append('./src/InverseProblems')
#sys.path.append('./src/utils')
from utils import * 
from plotting import *
from random_process import *
from model import *

In [6]:
# Extract test data for visualization or further processing
n_eig = 64
X = np.loadtxt('../../data/50-25-10/X_test_50resolution.csv', delimiter = ',')
y = np.loadtxt('../../data/50-25-10/y_test_50resolution.csv',delimiter = ',')

In [7]:
# Resolution parameters
resolution_h1 = (50, 50)
resolution_h2 = (25, 25)
resolution_h3 = (10, 10)

# PDE parameters
field_mean = 1
field_stdev = 1
lamb_cov = 0.1
mkl = 64

# Set up the model(s)
solver_h1 = Model(resolution_h1, field_mean, field_stdev, mkl, lamb_cov)
solver_h2 = Model(resolution_h2, field_mean, field_stdev, mkl, lamb_cov)
solver_h3 = Model(resolution_h3, field_mean, field_stdev, 32, lamb_cov)


# Adjust the trasmissivity based on h1
list1 = solver_h1.solver.mesh.coordinates()
list2 = solver_h2.solver.mesh.coordinates()
list3 = solver_h3.solver.mesh.coordinates()

# Convert lists to numpy arrays if they are not already
array1 = np.array(list1)
array2 = np.array(list2)
array3 = np.array(list3)

# Convert to structured arrays for easy row-wise comparison
dtype = {'names': ['f{}'.format(i) for i in range(array1.shape[1])],
         'formats': [array1.dtype] * array1.shape[1]}

structured_array1 = array1.view(dtype)
structured_array2 = array2.view(dtype)
structured_array3 = array3.view(dtype)

# Create the boolean vector by checking if each row in array1 is in array2
bool_vector2 = np.in1d(structured_array1, structured_array2)
bool_vector3 = np.in1d(structured_array1, structured_array3)

# Set the trasmissivity field
solver_h2.random_process.eigenvalues = solver_h1.random_process.eigenvalues
solver_h2.random_process.eigenvectors = solver_h1.random_process.eigenvectors[bool_vector2]

solver_h3.random_process.eigenvalues = solver_h1.random_process.eigenvalues
solver_h3.random_process.eigenvectors = solver_h1.random_process.eigenvectors[bool_vector3]

x_data = y_data = np.array([0.1, 0.3, 0.5, 0.7, 0.9])
datapoints = np.array(list(product(x_data, y_data)))

def solver_h1_data(x):
    solver_h1.solve(x)
    return solver_h1.get_data(datapoints)

def solver_h2_data(x):
    solver_h2.solve(x)
    return solver_h2.get_data(datapoints)

def solver_h3_data(x):
    solver_h3.solve(x)
    return solver_h3.get_data(datapoints)

In [12]:
n = 5000
start_0 = timeit.default_timer() 
fine_predictions = [solver_h1_data(X[1]) for i in range(n)]
end_0 = timeit.default_timer()

start_1 = timeit.default_timer() 
fine_predictions = [solver_h2_data(X[1]) for i in range(n)]
end_1= timeit.default_timer()

start_2 = timeit.default_timer() 
fine_predictions = [solver_h3_data(X[1]) for i in range(n)]
end_2= timeit.default_timer()


print('Time 50x50 resolution', (end_0-start_0)/n)
print('Time 25x25 resolution', (end_1-start_1)/n)
print('Time 10x10 resolution', (end_2-start_2)/n)
print('The 50/25 : ', (end_0 - start_0)/(end_1 - start_1) )
print('Time 50/10:',(end_0 - start_0)/(end_2 - start_2) )
print('Time 25/10:',(end_1 - start_1)/(end_2 - start_2) )

Time 50x50 resolution 0.011860543991799932
Time 25x25 resolution 0.00481061140820093
Time 10x10 resolution 0.0038268086416006555
The 50/25 :  2.465496167821947
Time 50/10: 3.0993303043339457
Time 25/10: 1.257081777203465


In [20]:
### LOW FIDELITY
# Choose the model parameters 
n_samples_lf = 64000
coeff_lf = 1e-09
# Choose the model parameters 
n_samples = 16000
coeff = 1e-08
# Initialize the neural network model
model_l = Sequential([
    Dense(256, input_shape=(X.shape[1],), activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(256, activation='gelu'),
    Dense(25, activation='linear')
])

model_l = load_model(f'models/model_25resolution_{n_samples_lf}samples_1_coeff{coeff_lf}.keras')


## MULTIFIDELITY 2STEP 
n_neurons = 256
# Initialize the neural network model
# Define the three branches of the model
input_params = Input(shape=(X.shape[1],))
input_pod = Input(shape=(y.shape[1],))

# Define the first branch (parameters)
x1 = Dense(n_neurons, activation='gelu')(input_params)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)
x1 = Dense(n_neurons, activation='gelu')(x1)

# Define the second branch (POD)
x2 = Dense(n_neurons, activation='gelu')(input_pod)

# # Define the second branch (POD)
# x3 = Dense(n_neurons, activation='gelu', kernel_regularizer=l2(w))(input_nn)

# Combine the outputs of the three branches
combined = Add()([x1,x2])
combined = Dense(n_neurons, activation='gelu')(combined)
output = Dense(25, activation='linear')(combined)

# Create the model
model_h = Model_nn(inputs=[input_params,input_pod], outputs=output)


model_h = load_model(f'models/model_2step_50-25resolution_{n_samples}samples_1_coeff{coeff}.keras')

model_lf = lambda input : model_l(input.reshape(1,64)).numpy().reshape(25)
model_hf = lambda input: model_h([input.reshape(1,64), model_l(input.reshape(1,64)).numpy()]).numpy().reshape(25)
model_hf1step = lambda input: model_h([input.reshape(1,64), solver_h2_data(input).reshape(1,25)]).numpy().reshape(25)

/Users/lucacaroselli/miniconda3/envs/fenics/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
start_0 = timeit.default_timer() 
fine_predictions = [model_lf(X[1]) for i in range(10000)]
end_0 = timeit.default_timer()

start_1 = timeit.default_timer() 
fine_predictions = [model_hf(X[1]) for i in range(10000)]
end_1= timeit.default_timer()

start_2 = timeit.default_timer() 
fine_predictions = [model_hf1step(X[1]) for i in range(10000)]
end_2= timeit.default_timer()

print('Time Low Fidelity', (end_0-start_0)/10000)
print('Time MF 2 step', (end_1-start_1)/10000)
print('Time MF 1 step', (end_2-start_2)/10000)
print('The 2Step/Low : ', (end_1 - start_1)/(end_0 - start_0) )
print('The 1Step/Low : ', (end_2 - start_2)/(end_0 - start_0) )
print('The 1Step/2Step: ', (end_2 - start_2)/(end_1 - start_1) )

Time Low Fidelity 0.004166737066699715
Time MF 2 step 0.009688190829100494
Time MF 1 step 0.011162845854200713
The 2Step/Low :  2.325126513628102
The 1Step/Low :  2.6790377399652674
The 1Step/2Step:  1.1522115997829838
